In [1]:
!pip install cvzone mediapipe opencv-python pillow

from cvzone.HandTrackingModule import HandDetector
import cv2
import os
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk

# Parameters
width, height = 1280, 720
gestureThreshold = 300
folderPath = "Presentation"

# Camera Setup
cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

# Hand Detector
detectorHand = HandDetector(detectionCon=0.8, maxHands=1)

# Variables
imgList = []
delay = 30
buttonPressed = False
counter = 0
drawMode = False
imgNumber = 0
delayCounter = 0
annotations = [[]]
annotationNumber = -1
annotationStart = False
hs, ws = int(120 * 1), int(213 * 1)  # width and height of small image

# Get list of presentation images
if not os.path.exists(folderPath):
    raise FileNotFoundError(f"Folder not found: {folderPath}")

pathImages = sorted([f for f in os.listdir(folderPath) if os.path.isfile(os.path.join(folderPath, f))])
if not pathImages:
    raise ValueError(f"No images found in folder: {folderPath}")

print(pathImages)

# Tkinter setup
root = tk.Tk()
root.title("Presentation")

# Create canvas for images
canvas1 = tk.Canvas(root, width=width, height=height)
canvas1.pack()

canvas2 = tk.Canvas(root, width=width, height=height)
canvas2.pack()

def update_image():
    global imgCurrent, img

    # Convert OpenCV image to PIL Image
    img_pil = Image.fromarray(cv2.cvtColor(imgCurrent, cv2.COLOR_BGR2RGB))
    img_tk = ImageTk.PhotoImage(image=img_pil)

    # Convert OpenCV image to PIL Image
    img_pil2 = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img_tk2 = ImageTk.PhotoImage(image=img_pil2)

    # Update the canvas with new image
    canvas1.create_image(0, 0, anchor=tk.NW, image=img_tk)
    canvas2.create_image(0, 0, anchor=tk.NW, image=img_tk2)

    # Keep reference to avoid garbage collection
    canvas1.image = img_tk
    canvas2.image = img_tk2

def handle_key(event):
    global buttonPressed
    if event.keysym == 'q':
        root.quit()

def main_loop():
    global buttonPressed, counter, annotations, annotationNumber, annotationStart, imgNumber, imgCurrent, img

    root.bind("<KeyPress>", handle_key)

    while True:
        # Get image frame
        success, img = cap.read()
        if not success:
            print("Failed to capture image")
            break

        img = cv2.flip(img, 1)
        pathFullImage = os.path.join(folderPath, pathImages[imgNumber])
        imgCurrent = cv2.imread(pathFullImage)
        
        if imgCurrent is None:
            print(f"Failed to load image: {pathFullImage}")
            continue

        # Find the hand and its landmarks
        hands, img = detectorHand.findHands(img)  # with draw
        # Draw Gesture Threshold line
        cv2.line(img, (0, gestureThreshold), (width, gestureThreshold), (0, 255, 0), 10)

        if hands and not buttonPressed:  # If hand is detected
            hand = hands[0]
            cx, cy = hand["center"]
            lmList = hand["lmList"]  # List of 21 Landmark points
            fingers = detectorHand.fingersUp(hand)  # List of which fingers are up

            # Constrain values for easier drawing
            xVal = int(np.interp(lmList[8][0], [width // 2, width], [0, width]))
            yVal = int(np.interp(lmList[8][1], [150, height-150], [0, height]))
            indexFinger = xVal, yVal

            if cy <= gestureThreshold:  # If hand is at the height of the face
                if fingers == [1, 0, 0, 0, 0]:
                    print("Left")
                    buttonPressed = True
                    if imgNumber > 0:
                        imgNumber -= 1
                        annotations = [[]]
                        annotationNumber = -1
                        annotationStart = False
                elif fingers == [0, 0, 0, 0, 1]:
                    print("Right")
                    buttonPressed = True
                    if imgNumber < len(pathImages) - 1:
                        imgNumber += 1
                        annotations = [[]]
                        annotationNumber = -1
                        annotationStart = False

            if fingers == [0, 1, 1, 0, 0]:
                cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)

            if fingers == [0, 1, 0, 0, 0]:
                if not annotationStart:
                    annotationStart = True
                    annotationNumber += 1
                    annotations.append([])
                print(annotationNumber)
                annotations[annotationNumber].append(indexFinger)
                cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)

            elif fingers == [0, 1, 1, 1, 0]:
                if annotations:
                    annotations.pop(-1)
                    annotationNumber -= 1
                    buttonPressed = True

        else:
            annotationStart = False

        if buttonPressed:
            counter += 1
            if counter > delay:
                counter = 0
                buttonPressed = False

        for i, annotation in enumerate(annotations):
            for j in range(len(annotation)):
                if j != 0:
                    cv2.line(imgCurrent, annotation[j - 1], annotation[j], (0, 0, 200), 12)

        imgSmall = cv2.resize(img, (ws, hs))
        h, w, _ = imgCurrent.shape
        imgCurrent[0:hs, w - ws: w] = imgSmall

        update_image()
        root.update_idletasks()
        root.update()

    cap.release()
    cv2.destroyAllWindows()
    root.destroy()

if __name__ == "__main__":
    main_loop()


['WhatsApp Image 2024-09-04 at 14.00.21_af9f1788.jpg', 'WhatsApp Image 2024-09-04 at 14.00.22_3dc0c015.jpg', 'WhatsApp Image 2024-09-04 at 14.00.22_c49e5b53.jpg', 'WhatsApp Image 2024-09-04 at 14.00.23_2647f2fa.jpg', 'WhatsApp Image 2024-09-04 at 14.00.24_97645509.jpg']


C:\Users\ROBIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Right
Left
Right
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
Right
Right
Right
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


RuntimeError: Too early to create image: no default root window